In [10]:
import csv
from matplotlib import pyplot as plt
import numpy as np
from scipy.misc import imread
import pickle
import cv2
from sklearn import linear_model
import seaborn as sns
import warnings
from jupyterthemes import jtplot
from IPython import display
import math

In [11]:
%matplotlib inline
jtplot.style()
sns.set_context("poster")
warnings.filterwarnings("ignore")

 A gaussian function is defined as: $$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-.5} \frac{(x-\mu)^2}{\sigma^2}$$

In [23]:
# a gaussian is described by the formula
def gaussian(mu, sigma_2, x):
    return 1/math.sqrt(2 * math.pi * sigma_2) * math.exp(-.5 * math.pow(x-mu,2)/sigma_2)
    

In [67]:
def gaussian_measurement_update(mu_1, sig_1, mu_2, sig_2):
    def mu_prime(sigma2, r2, mu, v):
        return 1/(sigma2 + r2) * (r2*mu + sigma2*v)

    def sigma_prime(sigma2, r2):
        return 1/(1/sigma2 + 1/r2)

    new_mean = mu_prime(sig_1, sig_2, mu_1, mu_2)
    new_var = sigma_prime(sig_1, sig_2)
    return new_mean, new_var

In [61]:
def plot_gaussian(mu, sigma_squared, label):
    plt.plot([gaussian(mu,sigma_squared,x) for x in np.arange(100)], label=label)

In [68]:
def plot_curves(mu_1, sig_1, mu_2, sig_2):
    plt.figure(figsize=(18,8))
    plot_gaussian(mu_1, sig_1, "Prior Distribution")
    plot_gaussian(mu_2, sig_2, "Sensed Distribution")
    update_mu, update_sig = gaussian_measurement_update(mu_1, sig_1, mu_2, sig_2)
    plot_gaussian(update_mu, update_sig, "Updated Distribution")
    plt.legend()

In [66]:
#scrub through thismanually for bad frames.... ouch time consuming
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from IPython.display import display


mu_1_slider= widgets.IntSlider(min=0,max=100,step=1,value=0)
sigma_1_slider= widgets.IntSlider(min=1,max=100,step=1,value=0)

mu_2_slider= widgets.IntSlider(min=0,max=100,step=1,value=0)
sigma_2_slider= widgets.IntSlider(min=1,max=100,step=1,value=0)


interactive_plot = interactive(plot_curves, mu_1=mu_1_slider, sig_1=sigma_1_slider,mu_2=mu_2_slider, sig_2=sigma_2_slider)
output = interactive_plot.children[-1]
output.layout.height = '450px'


widgets.VBox()
display(interactive_plot)

A Jupyter Widget

In [69]:
#here we are going to apply simple motion  model 
#mu_1, sig_1 are the current position an variance
# mu_2, sig_2 is the linear motion, e.g., move 4 places to the right with 
# variance of .5
def gaussian_preidict_update(mu_1, sig_1, mu_2, sig_2):
    new_mean = mu_1 + mu_2
    new_var = sig_1 + sig_2
    return new_mean, new_var